# Work with Language-Integrated Query (LINQ)

This tutorial teaches you features in .NET Core and the C# language. You’ll learn how to:

- Generate sequences with LINQ.
- Write methods that can be easily used in LINQ queries.
- Distinguish between eager and lazy evaluation.

You'll learn these techniques by building an application that demonstrates one of the basic skills of any magician: the [faro shuffle](https://en.wikipedia.org/wiki/Faro_shuffle). Briefly, a faro shuffle is a technique where you split a card deck exactly in half, then the shuffle interleaves each one card from each half to rebuild the original deck.

Magicians use this technique because every card is in a known location after each shuffle, and the order is a repeating pattern.

For your purposes, it is a light hearted look at manipulating sequences of data. The application you'll build constructs a card deck and then performs a sequence of shuffles, writing the sequence out each time. You'll also compare the updated order to the original order.

## Create a deck of cards

Commonly, a deck of playing cards has four suits, and each suit has thirteen values. Let's create two `enum` types to represent the suits and the ranks, and two extension methods that enumerate each suit and each rank:

In [3]:
public enum Suit
    {
        Clubs,
        Diamonds,
        Hearts,
        Spades
    }
    
Console.WriteLine(".NET Interactive knows about a Suit");

 public enum Rank
    {
        Two,
        Three,
        Four,
        Five,
        Six,
        Seven,
        Eight,
        Nine,
        Ten,
        Jack,
        Queen,
        King,
        Ace
    }

Console.WriteLine(".NET Interactive knows about a Face.");

A `Card` is defined by its `Suit` and its `Rank`:

In [5]:
public class Card
{
    public Suit Suit { get; }
    public Rank Rank { get; }

    public Card(Suit s, Rank r)
    {
        Suit = s;
        Rank = r;
    }

    public override string ToString() => $"{Rank} of {Suit}";
}

Console.WriteLine(".NET Interactive knows about a Card.");

Next, define a `Deck` class to represent the deck of cards. First, define two  [*iterator methods*](https://docs.microsoft.com/dotnet/csharp/iterators#enumeration-sources-with-iterator-methods) to generate the ranks and suits as sequences of values. These create the sequence of all suits and al ranks. They provide an easy way to enumerate all the possible combinations that make a deck of cards.

The `Deck` class has two constructors. The default constructor generate a deck in the sorted order. The second constructor takes a sequence of cards in the specified order. Later, you'll use the second constructor as you shuffle the deck.

In [6]:
static IEnumerable<Suit> Suits() => Enum.GetValues(typeof(Suit)) as IEnumerable<Suit>;
static IEnumerable<Rank> Ranks() => Enum.GetValues(typeof(Rank)) as IEnumerable<Rank>;

Console.WriteLine(".NET Interactive can generate suits and ranks.");

public class Deck
{
    private List<Card> cards;
    
    public Deck()
    {
        var sequence = from s in Suits()
                       from r in Ranks()
                       select new Card(s, r);
        cards = new List<Card>(sequence);
    }
    public Deck(IEnumerable<Card> cards) => this.cards = new List<Card>(cards);

    public IEnumerable<Card> Cards => cards;
}

Console.WriteLine(".NET Interactive knows about a Deck of cards");

The multiple `from` clauses produce a `SelectMany` call, which creates a single sequence from combining each element in the first sequence with each element in the second sequence. The order is important for our purposes. The first element in the first source sequence (Suits) is combined with every element in the second sequence (Ranks). This produces all thirteen cards of first suit. That process is repeated with each element in the first sequence (Suits). The end result is a deck of cards ordered by suits, followed by values.

It's important to keep in mind that whether you choose to write your LINQ in the query syntax used above or use method syntax instead, it's always possible to go from one form of syntax to the other. The above query written in query syntax can be written in method syntax as:

```csharp
var sequence = Suits().SelectMany(suit => Ranks().Select(rank => new Card(suit, rank));
```

The compiler translates LINQ statements written with query syntax into the equivalent method call syntax. Therefore, regardless of your syntax choice, the two versions of the query produce the same result. Choose which syntax works best for your situation: for instance, if you're working in a team where some of the members have difficulty with method syntax, try to prefer using query syntax.

## Display the deck

Now that you've created the deck of cards, you can define a method that displays the deck in your notebook. The following method declares a formatter that displays the deck of cards in 4 rows of 13. It uses the method `Enumerable.Range` to count from 0 through 3. Then, is uses the `Skip` and `Take` methods to skip rows previously displayed then take the next 13 cards. The images are returned from a storage location on Azure.

In [12]:
Formatter<Deck>.Register((deck, writer) =>
{
    PocketView v = div(
        Enumerable.Range(0,4).Select(row =>
            div(deck.Cards.Skip(row * 13).Take(13).Select(c =>
            {
                var face = Enum.GetName(typeof(Rank), c.Rank);
                var suit = Enum.GetName(typeof(Suit), c.Suit);
                return img[src:$"https://dotnetcards.azureedge.net/cards/{suit}{face}.png", width:64]();
            }))));
    writer.Write(v);
}, "text/html");

Console.WriteLine(".NET Interactive can display the deck of cards");

Deck d = new Deck();
display(d);

## Shuffle the deck

There's no shuffle method to take advantage of in the standard library, so you'll have to write your own. The shuffle method you'll be creating illustrates several techniques that you'll use with LINQ-based programs, so each part of this process will be explained in steps.

The first step in any good shuffle is to split the deck in two. The `Take%` and `Skip` methods that are part of the LINQ APIs provide that feature for you.

In order to add some functionality to how you interact with the `IEnumerable<Card>` sequence you'll get back from LINQ queries, you'll need to write some special kinds of methods called [extension methods](https://docs.microsoft.com/dotnet/csharp/programming-guide/classes-and-structs/extension-methods). Briefly, an extension method is a special purpose *static method* that adds new functionality to an already-existing type without having to modify the original type you want to add functionality to.

Look at the method signature for a moment, specifically the parameters:

```csharp
static IEnumerable<T> InterleaveSequenceWith<T> (this IEnumerable<T> first, IEnumerable<T> second)
```

You can see the addition of the `this` modifier on the first argument to the method. That means you call the method as though it were a member method of the type of the first argument. This method declaration also follows a standard idiom where the input and output types are `IEnumerable<T>`. That practice enables LINQ methods to be chained together to perform more complex queries.

Naturally, since you split the deck into halves, you'll need to join those halves together. In code, this means you'll be enumerating both of the sequences you acquired through `Take` and `Skip` at once, *`interleaving`* the elements, and creating one sequence: your now-shuffled deck of cards. Writing a LINQ method that works with two sequences requires that you understand how `IEnumerable<T>` works.

The `System.Collections.Generic.IEnumerable<T>` interface has one method: `GetEnumerator`. The object returned by `GetEnumerator` has a method to move to the next element, and a property that retrieves the current element in the sequence. You will use those two members to enumerate the collection and return the elements. This Interleave method will be an iterator method, so instead of building a collection and returning the collection, you'll use the `yield return` syntax shown above.

In [17]:
static IEnumerable<T> InterleaveSequenceWith<T> (this IEnumerable<T> first, IEnumerable<T> second)
{
    var firstIter = first.GetEnumerator();
    var secondIter = second.GetEnumerator();

    while (firstIter.MoveNext() && secondIter.MoveNext())
    {
        yield return firstIter.Current;
        yield return secondIter.Current;
    }
}

Console.WriteLine(".NET Interactive can shuffle the deck");

## Comparisons

How many shuffles it takes to set the deck back to its original order? To find out, you'll need to write a method that determines if two sequences are equal. After you have that method, you'll need to place the code that shuffles the deck in a loop, and check to see when the deck is back in order.

Writing a method to determine if the two sequences are equal should be straightforward. It's a similar structure to the method you wrote to shuffle the deck. Only this time, instead of `yield return`ing each element, you'll compare the matching elements of each sequence. When the entire sequence has been enumerated, if every element matches, the sequences are the same.

This shows a second LINQ idiom: terminal methods. They take a sequence as input (or in this case, two sequences), and return a single scalar value. When using terminal methods, they are always the final method in a chain of methods for a LINQ query, hence the name "terminal".

In [18]:
static bool SequenceEquals<T>(this IEnumerable<T> first, IEnumerable<T> second)
{
    var firstIter = first.GetEnumerator();
    var secondIter = second.GetEnumerator();

    while (firstIter.MoveNext() && secondIter.MoveNext())
    {
        if (!firstIter.Current.Equals(secondIter.Current))
        {
            return false;
        }
    }

    return true;
}
Console.WriteLine(".NET Interactive can compare sequences for equality.");

You can see this in action when you use it to determine when the deck is back in its original order. First, create and display the deck:

In [19]:
var deck = new Deck();
var deckDisplay = display(deck);

Put the shuffle code inside a loop, and stop when the sequence is back in its original order by applying the `SequenceEquals()` method. You can see it would always be the final method in any query, because it returns a single value instead of a sequence:

Run the next cell and take note of how the deck rearranges on each shuffle. After 8 shuffles (iterations of the do-while loop), the deck returns to the original configuration it was in when you first created it from the starting LINQ query.

In [20]:
int numberOfShuffles = 0;
var shuffleDisplay = display($"Number of shuffles: {numberOfShuffles}");
deckDisplay = display(deck);
var shuffle = deck;
do
{
    await Task.Delay(3000);
    shuffle = new Deck(shuffle.Cards.Take(26).InterleaveSequenceWith(shuffle.Cards.Skip(26)));
    numberOfShuffles++;
    deckDisplay.Update(shuffle);
    shuffleDisplay.Update($"Number of shuffles: {numberOfShuffles}");
} while(!deck.Cards.SequenceEquals(shuffle.Cards));
Console.WriteLine("Done!");

## Experiment

The sample you've built so far executes an *out shuffle*, where the top and bottom cards stay the same on each run. Let's make one change: we'll use an *in shuffle* instead, where all 52 cards change position. For an in shuffle, you interleave the deck so that the first card in the bottom half becomes the first card in the deck. That means the last card in the top half becomes the bottom card. This is a simple change to a singular line of code. Update the current shuffle query by switching the positions of `Take` and `Skip`. This will change the order of the top and bottom halves of the deck:

```csharp
shuffle = shuffle.Skip(26).InterleaveSequenceWith(shuffle.Take(26));
```

Run the previous cell again, and you'll see that it takes 52 iterations for the deck to reorder itself.

## Conclusion

In this project, you covered:

- using LINQ queries to aggregate data into a meaningful sequence
- writing Extension methods to add our own custom functionality to LINQ queries
- locating areas in our code where our LINQ queries might run into performance issues like degraded speed
- lazy and eager evaluation in regards to LINQ queries and the implications they might have on query performance

Aside from LINQ, you learned a bit about a technique magicians use for card tricks. Magicians use the Faro shuffle because they can control where every card moves in the deck. Now that you know, don't spoil it for everyone else!

For more information on LINQ, see:

- [Language Integrated Query (LINQ)](https://docs.microsoft.com/dotnet/csharp/programming-guide/concepts/linq/index)
- [Introduction to LINQ](https://docs.microsoft.com/dotnet/csharp/programming-guide/concepts/linq/index)
- [Basic LINQ Query Operations (C#)](https://docs.microsoft.com/dotnet/csharp/programming-guide/concepts/linq/basic-linq-query-operations)
- [Data Transformations With LINQ (C#)](https://docs.microsoft.com/dotnet/csharp/programming-guide/concepts/linq/data-transformations-with-linq)
- [Query Syntax and Method Syntax in LINQ (C#)](https://docs.microsoft.com/dotnet/csharp/programming-guide/concepts/linq/query-syntax-and-method-syntax-in-linq)
- [C# Features That Support LINQ](https://docs.microsoft.com/dotnet/csharp/programming-guide/concepts/linq/features-that-support-linq)
